In [1]:
import pandas as pd
import re
import requests

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
df = pd.read_csv('../raw_data/database_entities.csv')
df

,Unnamed: 0,entity,score,index,word,start,end,line_id
0,0,B-PER,0.915520,4,V,11,12,0
1,1,B-LOC,0.999274,17,American,51,59,0
2,2,I-LOC,0.998381,18,Samoa,60,65,0
3,3,B-MISC,0.999530,29,Vietnamese,104,114,0
4,4,B-ORG,0.973456,46,Tour,196,200,0
...,...,...,...,...,...,...,...,...
15967,15967,I-LOC,0.996284,223,##z,970,971,1357
15968,15968,B-LOC,0.999348,284,A,1227,1228,1357
15969,15969,I-LOC,0.874068,285,##gade,1228,1232,1357
15970,15970,I-LOC,0.987688,286,##z,1232,1233,1357


In [4]:
df = df.drop(columns = ['Unnamed: 0', 'index', 'start', 'end'])
df

,entity,score,word,line_id
0,B-PER,0.915520,V,0
1,B-LOC,0.999274,American,0
2,I-LOC,0.998381,Samoa,0
3,B-MISC,0.999530,Vietnamese,0
4,B-ORG,0.973456,Tour,0
...,...,...,...,...
15967,I-LOC,0.996284,##z,1357
15968,B-LOC,0.999348,A,1357
15969,I-LOC,0.874068,##gade,1357
15970,I-LOC,0.987688,##z,1357


In [5]:
df = df[(df["entity"]=="B-LOC")|(df["entity"]=="I-LOC")][df["score"]>0.98]

/var/folders/rv/1874f2853zqf1lx4mr_wkbsc0000gn/T/ipykernel_28372/4092738895.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(df["entity"]=="B-LOC")|(df["entity"]=="I-LOC")][df["score"]>0.98]


In [6]:
df.rename(columns={"line_id": "victim_id"})

,entity,score,word,victim_id
1,B-LOC,0.999274,American,0
2,I-LOC,0.998381,Samoa,0
7,B-LOC,0.999507,US,0
9,B-LOC,0.999251,American,0
10,I-LOC,0.998292,Samoa,0
...,...,...,...,...
15965,B-LOC,0.999359,A,1357
15967,I-LOC,0.996284,##z,1357
15968,B-LOC,0.999348,A,1357
15970,I-LOC,0.987688,##z,1357


In [7]:
r=list(df['word'].unique())

In [8]:
s=list()
for i in r:
    if len(re.findall("#.*",i))==0:
        s.append(i)
    else:
        pass
s    

['American',
 'Samoa',
 'US',
 'Col',
 'Sri',
 'Lanka',
 'Lebanon',
 'Beirut',
 'Allah',
 'Ka',
 'N',
 'G',
 'K',
 'Cameroon',
 'United',
 'States',
 'America',
 'Minnesota',
 'of',
 'Haiti',
 'Port',
 '-',
 'au',
 'Prince',
 'Mexico',
 'Arizona',
 'Florida',
 'No',
 'Ma',
 'Bar',
 'Ram',
 'Son',
 'Uttar',
 'Pradesh',
 'India',
 'San',
 'Niger',
 'Mali',
 'Egypt',
 'Paris',
 'France',
 'Myrtle',
 'Beach',
 'SC',
 'Miami',
 'FL',
 'Houston',
 'TX',
 'Diego',
 'CA',
 'New',
 'Orleans',
 'LA',
 'Scott',
 'A',
 'Chicago',
 'IL',
 'Columbus',
 'OH',
 'Denver',
 'CO',
 'Silver',
 'Spring',
 'England',
 'Koch',
 'Nigeria',
 'Europe',
 'South',
 'Africa',
 'Johannesburg',
 'B',
 'Be',
 'Pieter',
 'Hill',
 'Los',
 'Angeles',
 'U',
 'S',
 '.',
 'Indonesia',
 'York',
 'City',
 'NY',
 'Mom',
 'Southern',
 'Boston',
 'Philippines',
 'California',
 'North',
 'Korea',
 'China',
 'Tu',
 'River',
 'USA',
 'Beijing',
 'Guangdong',
 'Province',
 'To',
 'Ko',
 'Ghana',
 'Northern',
 'Sudan',
 'Ukraine',
 

In [9]:
canc=list()
for i in s:
    if len(i)==1:
        canc.append(i)
    try:
        int(i)
        canc.append(i)
    except:
        pass
    
        

In [10]:
def transform(item):
    if item in canc:
        item=item.replace(item,"np.nan")
    return item
df["word"]=df["word"].apply(transform)      

In [11]:
import numpy as np
df["word"]=df["word"].replace("#.*",np.nan,regex=True)

In [12]:
df=df.dropna(subset=["word"])

In [13]:
df=df[df["word"]!="np.nan"]

In [14]:
df.drop(columns="score",inplace=True)

In [15]:
df.drop(columns="entity",inplace=True)

In [16]:
df.rename(columns={"word":"location", "line_id":"victim_index"},inplace=True)

In [17]:
df

,location,victim_index
1,American,0
2,Samoa,0
7,US,0
9,American,0
10,Samoa,0
...,...,...
15958,Libya,1356
15959,Benin,1357
15960,City,1357
15964,Niger,1357


In [18]:
df.to_csv('victims_locations.csv')

In [19]:
coordinates = {}
for i in df['location'].unique():
    params = {'access_key':'f4fb3cb9d8780b5c1fd36dc5e846523b', 'query': i}
    response = requests.get('http://api.positionstack.com/v1/forward',params=params).json()
    print(i)
    if 'error' in response.keys():
        pass
    if 'data' in response.keys():
        if len(response['data']) ==0:
            pass
    
        try:
            latitude = response["data"][0]["latitude"]
            longitude = response["data"][0]["longitude"]
            country = response["data"][0]["country"]
            country_code = response["data"][0]["country_code"]
            coordinates[i] = [latitude, longitude, country, country_code]
        except TypeError:
            print(response)

    
    
    

American
Samoa
US
Col
Sri
Lanka
Lebanon
Beirut
Allah
Ka
Cameroon
United
States
America
Minnesota
of
Haiti
Port
au
Prince
Mexico
Arizona
Florida
No
Ma
Bar
Ram
Son
Uttar
Pradesh
India
San
Niger
Mali
Egypt
Paris
France
Myrtle
Beach
SC
Miami
FL
Houston
TX
Diego
CA
New
Orleans
LA
Scott
Chicago
IL
Columbus
OH
Denver
CO
Silver
Spring
England
Koch
Nigeria
Europe
South
Africa
Johannesburg
Be
Pieter
Hill
Los
Angeles
Indonesia
York
City
NY
Mom
Southern
Boston
Philippines
California
North
Korea
China
Tu
River
USA
Beijing
Guangdong
Province
To
Ko
Ghana
Northern
Sudan
Ukraine
Virginia
Kiev
Washington
Detroit
Lia
Jersey
Atlantic
Philadelphia
Mississippi
Cambodia
Thailand
Me
Mumbai
Belgium
UK
Japan
Hong
Kong
Singapore
Ten
Spain
Madrid
London
Coventry
Romania
Birmingham
Kabul
Hungary
Sierra
Leone
Ecuador
Dominican
Republic
Kansas
Portland
Honduras
Fresno
Belize
Central
Place
Texas
Nevada
Oregon
DC
Street
Ireland
Belfast
Lithuania
Italy
Lagos
Qatar
Gulf
Nepal
Ball
Scotland
Se
Bangladesh
Pedro
Sul
Austra

In [20]:
coordinates

{'American': [40.769013, -84.192828, 'United States', 'USA'],
 'Samoa': [-13.63914, -172.438241, 'Samoa', 'WSM'],
 'Col': [3.420497, -73.166783, 'Colombia', 'COL'],
 'Sri': [32.93643, 74.399, 'Pakistan', 'PAK'],
 'Lanka': [7.622919, 80.68822, 'Sri Lanka', 'LKA'],
 'Lebanon': [34.095821, 35.95655, 'Lebanon', 'LBN'],
 'Beirut': [33.88432, 35.501527, 'Lebanon', 'LBN'],
 'Allah': [13.51238, 7.60735, 'Niger', 'NER'],
 'Cameroon': [4.585076, 12.473672, 'Cameroon', 'CMR'],
 'United': [37.99066, -81.4265, 'United States', 'USA'],
 'States': [39.715956, -96.999668, 'United States', 'USA'],
 'America': [39.715956, -96.999668, 'United States', 'USA'],
 'Minnesota': [46.63636, -94.579641, 'United States', 'USA'],
 'Haiti': [19.251299, -72.294701, 'Haiti', 'HTI'],
 'Port': [33.461769, -117.701464, 'United States', 'USA'],
 'Prince': [46.578691, -63.966903, 'Canada', 'CAN'],
 'Mexico': [19.371422, -99.150344, 'Mexico', 'MEX'],
 'Arizona': [34.255149, -111.62621, 'United States', 'USA'],
 'Florida': 

In [21]:
df2=pd.DataFrame.from_dict(coordinates,orient='index')

In [22]:
df2.reset_index(inplace = True)

In [23]:
df2.to_csv('victims_locations2.csv')

In [24]:
df2

,index,0,1,2,3
0,American,40.769013,-84.192828,United States,USA
1,Samoa,-13.639140,-172.438241,Samoa,WSM
2,Col,3.420497,-73.166783,Colombia,COL
3,Sri,32.936430,74.399000,Pakistan,PAK
4,Lanka,7.622919,80.688220,Sri Lanka,LKA
...,...,...,...,...,...
490,Tripoli,32.892500,13.180012,Libya,LBY
491,Malta,35.888972,14.431432,Malta,MLT
492,Ara,25.552353,84.666138,India,IND
493,Sabha,27.037660,14.428320,Libya,LBY
